In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegressionCV
from fastai.column_data import *

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
PATH = "data/hike"

In [3]:
!ls {PATH}

sample_submission_only_headers.csv  test.zip  train.csv  user_features.csv


In [4]:
#import patoolib  #uncomment this if running for 1st time
#patoolib.extract_archive(f'{PATH}/train.zip',outdir=PATH) #uncomment this if running for 1st time

In [5]:
#!ls {PATH}

In [6]:
dtype = {'node1_id':'int32','node2_id':'int32','is_chat':'int8','node_id':'int32','id':'int32','f1':'int8','f2':'int8','f3':'int8',
         'f4':'int8','f5':'int8','f6':'int8','f7':'int8','f8':'int8','f9':'int8','f10':'int8','f11':'int8','f12':'int8','f13':'category',}

In [7]:
#  %%time
# df_train = pd.read_csv(f'{PATH}/train.csv', low_memory=True,dtype=dtype)

In [8]:
# %%time
# user_feat = pd.read_csv(f'{PATH}/user_features.csv', low_memory=True,dtype=dtype)

In [10]:
# user_feat.head()

In [9]:
# ##feature engineering
# #col = ['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12']
# def add_feature(df):
#     #for col in ['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12']:
#     df['day_avg'] = df.iloc[:,1:13].mean(axis=1).astype('float32')
#     df['day_std']  = df.iloc[:,1:12].std (axis=1).astype('float32')
#     df['day_max'] = df.iloc[:,1:12].max(axis=1).astype('int8')
#     df['day_min']  = df.iloc[:,1:12].min (axis=1).astype('int8')
#     df['day_range'] = df.iloc[:,1:12].max(axis=1) - df.iloc[:,1:12].min(axis=1).astype('int8')
#     df['day_maxmin'] = df.iloc[:,1:12].max(axis=1) / df.iloc[:,1:12].min(axis=1).astype('float32')
#     return df

In [9]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)

In [11]:
# user_feat = add_feature(user_feat)

In [12]:
# user_feat['day_maxmin'].fillna(0,inplace=True)
# user_feat['day_maxmin'].replace(-np.inf,0,inplace=True)
# user_feat['day_maxmin'].replace(np.inf,0,inplace=True)

In [33]:
# user_feat.dtypes

In [32]:
#  display_all(df_train.head().T)

In [22]:
# df_train0 = df_train[df_train['is_chat']==0]
# df_train1 = df_train[df_train['is_chat']==1]

In [11]:
# df_train0.shape, df_train1.shape

((68410018, 3), (2251784, 3))

In [ ]:
#df_train0.head()

In [13]:
# df_train.shape, user_feat.shape

((70661802, 3), (8264276, 14))

In [9]:
# df_train.sort_values('is_chat',axis=1, ascending=False)[0:3000000].groupby(['is_chat'])['is_chat'].count()/3000000

In [23]:
# df_train = pd.concat([df_train1,df_train0],ignore_index=True)

In [24]:
# del(df_train0,df_train1)

In [34]:
# df_train.nunique()

In [35]:
# user_feat.nunique()

In [27]:
# df_test = pd.read_csv(f'{PATH}/test.zip', low_memory=True,dtype=dtype)

In [10]:
#df_train.tail()

In [11]:
#user_feat.tail()

In [13]:
#df_train.shape, user_feat.shape, df_test.shape

In [11]:
# display_all(df_train.head().T)

In [15]:
#user_feat.describe()

In [16]:
#df_test.describe()

In [17]:
#df_train.groupby('is_chat')['is_chat'].count()

In [13]:
# df_train = df_train.merge(user_feat, left_on='node2_id', right_on='node_id').drop(['node_id'],axis=1)
# no_rename = {'node1_id', 'node2_id','is_chat'}
# df_train.columns = ['{}{}'.format(c, '' if c in no_rename else '_2') for c in df_train.columns]
# df_train = df_train.merge(user_feat, left_on='node1_id', right_on='node_id').drop(['node_id'],axis=1)

In [17]:
# df_train.to_feather('tmp/df_train_hike')

In [28]:
# df_test = df_test.merge(user_feat, left_on='node2_id', right_on='node_id').drop(['node_id'],axis=1)
# no_rename = {'node1_id', 'node2_id','id'}
# df_test.columns = ['{}{}'.format(c, '' if c in no_rename else '_2') for c in df_test.columns]
# df_test = df_test.merge(user_feat, left_on='node1_id', right_on='node_id').drop(['node_id'],axis=1)

In [30]:
#display_all(df_test.head().T)

In [31]:
# df_test.to_feather('tmp/df_test_hike')

In [9]:
df_train = pd.read_feather('tmp/df_train_hike')
df_test = pd.read_feather('tmp/df_test_hike')

/usr/local/lib/python3.6/dist-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [20]:
df_test.head()

,id,node1_id,node2_id,f1_2,f2_2,f3_2,f4_2,f5_2,f6_2,f7_2,...,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
0,1,7107094,8010772,31,24,2,31,24,3,31,...,31,6,1,31,8,1,31,9,1,15
1,1514667,7107094,7110578,5,5,1,5,5,1,5,...,31,6,1,31,8,1,31,9,1,15
2,10094767,7107094,3504535,31,16,1,31,11,0,31,...,31,6,1,31,8,1,31,9,1,15
3,5262085,7107094,4911208,31,22,6,31,26,5,31,...,31,6,1,31,8,1,31,9,1,15
4,3434399,1242529,8010772,31,24,2,31,24,3,31,...,31,18,0,31,19,0,31,17,1,15


In [14]:
df_train.shape, df_test.shape

((70661802, 41), (11776968, 41))

In [22]:
#df_train.dtypes, df_test.dtypes

In [10]:
df, y = df_train.drop(['is_chat'],axis=1), df_train.is_chat

In [11]:
from sklearn.model_selection import train_test_split
def split_vals(a,n): return a[:n].copy(), a[n:].copy()
# n_valid = 10000000
# n_trn = len(df)-n_valid
# #raw_train, raw_valid = split_vals(df_raw, n_trn)
# X_train, X_valid = split_vals(df, n_trn)
# y_train, y_valid = split_vals(y, n_trn)
X_train, X_valid, y_train, y_valid = train_test_split(df, y, test_size=0.20, random_state=42)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((56529441, 40), (56529441,), (14132361, 40), (14132361,))

In [10]:
def print_score(m):
    res = [roc_auc_score(y_train,m.predict_proba(X_train)[:,1]), roc_auc_score(y_valid,m.predict_proba(X_valid)[:,1]),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [12]:
test_id = df_test.id
df_test = df_test.drop(['id'],axis =1)

In [10]:
set_rf_samples(1000000)

In [40]:
# %%time
# m = RandomForestClassifier(n_estimators=5, n_jobs=-1,min_samples_leaf=2, oob_score=True)
# m.fit(X_train, y_train)
# print_score(m)

[0.6918802183399206, 0.6904179209723427, 0.9701661397991149, 0.95329775, 0.9701111466026905]
CPU times: user 4min 2s, sys: 33.9 s, total: 4min 36s
Wall time: 2min 17s


In [61]:
# preds_prob = m.predict_proba(df_test)[:,1]
# preds = preds_prob.squeeze()
# submit = pd.DataFrame({'id':test_id, 'is_chat':preds}, columns=['id', 'is_chat'])
# submit.to_csv('submit_1.csv',index=False)

In [65]:
submit.shape

(11776968, 2)

In [16]:
%%time
m = RandomForestClassifier(n_estimators=50, n_jobs=-1,min_samples_leaf=3, oob_score=True,random_state=1)
%time m.fit(X_train[0:2000000], y_train[0:2000000])
roc_auc_score(y_valid,m.predict_proba(X_valid)[:,1])

CPU times: user 7min, sys: 23.2 s, total: 7min 23s
Wall time: 2min 42s


0.8097364975358665

In [16]:
#y_valid[0:100]

In [22]:
#roc_auc_score(y_valid[0:100],m.predict_proba(X_valid[0:100])[:,1])

In [17]:
preds_prob = m.predict_proba(df_test)[:,1]
preds = preds_prob.squeeze()
submit = pd.DataFrame({'id':test_id, 'is_chat':preds}, columns=['id', 'is_chat'])
submit.to_csv('submit_9.csv',index=False)

In [9]:
# m = XGBClassifier(random_stae=1,n_jobs=-1,n_estimators=20)
# %time m.fit(X_train[0:300000], y_train[0:300000])
# %time print_score(m)

CPU times: user 8.61 s, sys: 88 ms, total: 8.7 s
Wall time: 2.79 s


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[0.7980983816117887, 0.7973220789326988, 0.9711113545403873, 0.953573]
CPU times: user 2min 24s, sys: 55.9 s, total: 3min 20s
Wall time: 2min 18s


In [13]:
# m = XGBClassifier(random_stae=1,n_jobs=-1,n_estimators=20,learning_rate=0.3)
# %time m.fit(X_train[0:300000], y_train[0:300000])
# %time print_score(m)

CPU times: user 8.65 s, sys: 100 ms, total: 8.75 s
Wall time: 2.78 s


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[0.826527643414976, 0.82474018290218, 0.9711102635408302, 0.9535721666666667]
CPU times: user 2min 28s, sys: 55.5 s, total: 3min 24s
Wall time: 2min 23s


In [14]:
# m = XGBClassifier(random_stae=1,n_jobs=-1,n_estimators=20,learning_rate=0.3,max_depth=5)
# %time m.fit(X_train[0:300000], y_train[0:300000])
# %time print_score(m)

CPU times: user 14.2 s, sys: 160 ms, total: 14.4 s
Wall time: 4.51 s


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[0.831535705204016, 0.8298450590613586, 0.9710962169215327, 0.9535643333333333]
CPU times: user 3min 23s, sys: 56.2 s, total: 4min 19s
Wall time: 2min 42s


In [17]:
X_train[['f13','f13_2']] = X_train[['f13','f13_2']].astype('int8')
X_valid[['f13','f13_2']] = X_valid[['f13','f13_2']].astype('int8')

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [18]:
df_test[['f13','f13_2']] = df_test[['f13','f13_2']].astype('int8')

In [22]:
#m = XGBClassifier(random_stae=1,n_jobs=-1,n_estimators=90,learning_rate=0.3,max_depth=5)
#%time m.fit(X_train[0:25000000], y_train[0:25000000])
#%time print_score(m)
roc_auc_score(y_valid,m.predict_proba(X_valid)[:,1])

0.8472517104725452

In [23]:
preds_prob = m.predict_proba(df_test)[:,1]
preds = preds_prob.squeeze()
submit = pd.DataFrame({'id':test_id, 'is_chat':preds}, columns=['id', 'is_chat'])
submit.to_csv('submit_9.csv',index=False)

In [26]:
#FileLink(submit)

In [27]:
# m = LogisticRegressionCV(random_state=1,n_jobs=-1,cv=5,penalty='l1',solver='liblinear')
# %time m.fit(X_train[0:3000000], y_train[0:3000000])
# %time print_score(m)

CPU times: user 3min 4s, sys: 2.35 s, total: 3min 6s
Wall time: 1h 5min 51s
[0.7532771602622153, 0.7444760333352842, 0.9711113545403873, 0.953573]
CPU times: user 47.1 s, sys: 26.7 s, total: 1min 13s
Wall time: 1min 8s


In [28]:
# preds_prob = m.predict_proba(df_test)[:,1]
# preds = preds_prob.squeeze()
# submit = pd.DataFrame({'id':test_id, 'is_chat':preds}, columns=['id', 'is_chat'])
# submit.to_csv('submit_4.csv',index=False)